In [1]:
from algo.utils.db import core_query
import warnings
warnings.filterwarnings('ignore')

def show(df, cols_to_blank):
    s = df.head(3)
    for c in cols_to_blank:
        s[c] = ''
    return s

### Get Click and Imp data for Ad Copies in Campaigns that have both DKI and Non DKI Ads competing

In [2]:
# All basecampaigns created in the past 180 days
tax = core_query('''
                SELECT DISTINCT
                    csc.name     AS sc_name,
                    csc.id       AS sc_id,
                    cbc.id       AS bc_id,
                    bac.id       AS ac_id,
                    bac.headline AS headline,
                    cbc.campaigntype,
                    cbc.target
                FROM analytics.client_static_basic a
                    JOIN control.campaignconfiguration ccc
                        ON a.adconfiguration_id = ccc.adconfiguration_id
                             AND a.client_type = 'Local'
                    JOIN control.servicecategory csc
                        ON ccc.servicecategory_id = csc.id
                    JOIN control.basecampaign cbc
                        ON ccc.id = cbc.campaignconfiguration_id
                    JOIN control.baseadcopy bac
                        ON cbc.id = bac.campaign_id
                             AND headline IS NOT NULL
                    WHERE cbc.createddate >= current_date - 10
                ORDER BY cbc.id
                '''
                )

In [3]:
show(tax, ['headline', 'sc_name'])

,sc_name,sc_id,bc_id,ac_id,headline,campaigntype,target
0,,5666765240,6835018132,6835018443,,0,4
1,,5666765240,6835018132,6835018444,,0,4
2,,5666765240,6835018132,6835018445,,0,4


In [4]:
# Keep only Basecampaigns that have had both DKI and Non DKI Ad's created
tax['dki'] = [1 if '{' in x else 0 for x in tax.headline]
tax['nodki'] = 1 - tax.dki
tax = tax.groupby('bc_id').filter(lambda x: sum(x.dki) > 0 and sum(x.nodki) > 0)
tax['dki'] = ['YES' if x==1 else 'NO' for x in tax.dki]
tax.drop('nodki', axis=1, inplace=True)

In [5]:
show(tax, ['headline', 'sc_name'])

,sc_name,sc_id,bc_id,ac_id,headline,campaigntype,target,dki
3648,,230907388,6835094160,6835094342,,3,0,YES
3649,,230907388,6835094160,6835094343,,3,0,NO
3650,,230907388,6835094160,6835094344,,3,0,NO


In [6]:
# All ad copy data in the past 10 days for these campaigns
data = core_query('''
                SELECT
                    baseadcopy_id        AS ac_id,
                    sum(agg.impressions) AS imps,
                    sum(agg.clicks)      AS clicks
                FROM aggregates.provider_reports_baseadcopy_day agg
                WHERE day >= current_date - 10
                            AND agg.baseadcopy_id IN {parameter}
                GROUP BY baseadcopy_id
                '''.format(parameter = tuple(set(tax.ac_id)))
               )

In [7]:
# Initial DF-> merge in clicks and impressions to each ad copy
df = pd.merge(tax, data, on='ac_id', how='left').fillna(0)
show(df, ['sc_name', 'headline', 'imps', 'clicks'])

,sc_name,sc_id,bc_id,ac_id,headline,campaigntype,target,dki,imps,clicks
0,,230907388,6835094160,6835094342,,3,0,YES,,
1,,230907388,6835094160,6835094343,,3,0,NO,,
2,,230907388,6835094160,6835094344,,3,0,NO,,


### Only keep Service Categories meeting a minimum threshold for total DKI and Non DKI impressions

In [8]:
# Dictionary of SC_ID to SC_Name
dSC = {k: v for k, v in zip(df.sc_id, df.sc_name)}

# Group data by Service Category, Keep only Service Categories with >=1000 DKI and >=1000 NonDKI Impressions
df = df.groupby(['sc_id', 'dki'], as_index=False)['imps', 'clicks'].sum()
scs_keep = df[df.imps>1000].groupby('sc_id').filter(lambda x: len(x.imps)==2).sc_id.drop_duplicates().tolist()
df = df[df.sc_id.isin(scs_keep)]

### For each Service Category, how does the CTR for DKI vs. Non DKI Ads compare?

In [29]:
# Calculate CTR, Pivot Table for DKI and NonDKI Imps and CTR
df['ctr'] = df.clicks / df.imps
df = pd.pivot_table(df, index='sc_id', columns='dki', values=['ctr', 'imps'], aggfunc=np.sum)
cols = list(zip(df.columns.get_level_values(1).tolist(), df.columns.get_level_values(0).tolist()))
df.columns = [x[0]+'_'+x[1] for x in cols]
df.reset_index(inplace=True)

# Calculate % difference DKI to NonDKI
df['dki_vs_nodki'] = (df.YES_ctr - df.NO_ctr) / df.NO_ctr
df.sort_values('dki_vs_nodki', inplace=True)

# Final Dataframe for SC level DKI Analysis
df['sc_name'] = df.sc_id.map(dSC)
dki_data = df.copy()

show(dki_data, ['NO_imps', 'YES_imps', 'sc_name'])

,sc_id,NO_ctr,YES_ctr,NO_imps,YES_imps,dki_vs_nodki,sc_name
1,232927053,0.014093,0.022352,,,0.586006,
0,60330,0.012087,0.026589,,,1.199784,
2,1964749030,0.000476,0.001159,,,1.436848,


### For each Service Category, what does the keyphrase taxonomy look like? Can we gain any insight by understanding the types of keywords that high DKI CTR campaigns have vs the types of keywords that low DKI CTR campaigns have?

In [23]:
# Grab keyphrases and impressions/clicks for SC's of interest
tax_kp = core_query('''
                SELECT
                    ccc.servicecategory_id AS sc_id,
                    k.phrase               AS keyphrase,
                    sum(agg.impressions)   AS imps,
                    sum(agg.clicks)        AS clicks
                FROM control.campaignconfiguration ccc
                    JOIN control.basecampaign cbc
                        ON ccc.id = cbc.campaignconfiguration_id
                             AND ccc.servicecategory_id IN {parameter}
                JOIN control.listing l ON cbc.id = l.campaign_id
                JOIN control.keyphrase k ON l.keyphraseid = k.id
                JOIN aggregates.provider_reports_listing_day agg ON l.id = agg.listing_id
                    AND agg.day>= CURRENT_DATE - 3
                GROUP BY ccc.servicecategory_id, k.phrase
                '''.format(parameter = tuple(set(dki_data.sc_id)))
              )

tax_kp['sc_name'] = tax_kp.sc_id.map(dSC)

tax_kp.sort_values(['sc_id', 'imps'], ascending=False, inplace=True)

# Final Dataframe for all Keyphrases in SC's in interest, along with their clicks and impressions
dki_tax = tax_kp.copy()

In [24]:
show(dki_tax, ['NO_imps', 'YES_imps', 'sc_name'])

,sc_id,keyphrase,imps,clicks,sc_name
0,60330,,,,
1,60330,,,,
2,60330,,,,


In [27]:
path = ''
filename = ''

f = path + filename + '.xls'
writer = pd.ExcelWriter(f)

dki_data.to_excel(writer, 'SC_Data')
dki_tax.to_excel(writer, 'SC_Taxonomy')

writer.save()